# Inglise keele tõlkevastete kandidaatide saniteerimine

_Esta Prangel_

## Sissejuhatus

Andmestik on pärit sõnastikest ja paralleelkorpustest.

__Sõnastikud__:
- TEA
- Silvet
- Majandus
- Indrek Hein
- K-dictionaries

__Korpused__:
- Professionaalsed tõlkemälud
- Avalikud paralleelkorpused

Andmestikust on kaks versiooni: 
1. `ekilex_import_data_itermax.json`
2. `ekilex_import_data_inter.json`

Erinevus on korpuste parsimisel kasutatud joondamismetoodikas. Kõik kõrgema kaaluga kandidaadid on neis samad, aga korpustest tulevad variandid võivad erineda.

Kaustas “Keelevaraga” on andmestikesse lisatud sõnastikest pärit andmed. Kui vaste kaal on suurem kui 0,9, on see pärit sõnaraamatust. Millisest sõnaraamatust, on võimalik öelda ainult siis, kui tegu on Kdictionaryga. Teiste puhul pole meil võtta definitsioone ega kasutusnäiteid ja kuna allikaviide käib nende, mitte vaste enda kohta, siis allikaviidet pole kuhugi panna. Nii et kui vaste kaal on suurem kui 0,9 ja see pole pärit Kdictionaryst, on see pärit Indrek Heina ümberpööratud sõnastikust, Tea sõnastikest, majandussõnastikust või Silveti sõnastikust.

Näidissõnastik, millesugustest andmestik koosneb:

```json
{
    "headwordValue": "varsseller",
    "headwordLang": "est",
    "synCandidateDatasetCode": "ingitermax",
    "synCandidateWords": [
      {
        "value": "celery",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "ribbed",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "piece",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "matchstick",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "asparagus",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "vegetable",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      },
      {
        "value": "stalk",
        "lang": "eng",
        "weight": 0.8,
        "posCodes": [],
        "definitions": [],
        "usages": []
      }
    ]
  }

## Andmete saniteerimine

### Andmestikud

Koodi käivitamisel tuleb valida, kumma joondamismetoodikaga koostatud andmestikku töötlema hakatakse.

In [433]:
base_data = 'failid/andmestik/keelevaraga/ekilex_import_data_inter.json'
#base_data = 'failid/andmestik/keelevaraga/ekilex_import_data_inter_051023_100000.json'
#base_data = 'failid/andmestik/keelevaraga/ekilex_import_data_itermax.json'

Vajalik on loetelu ÜSi keelenditest.

In [434]:
ys = 'failid/andmestik/YS/_SELECT_DISTINCT_word_value_FROM_lexeme_JOIN_word_ON_lexeme_word_202310051024.csv'

### Vajalike teekide import

In [435]:
import csv
import string
import json
import os
import random
import pandas as pd

### Üleliigsete eesti keelendite eemaldamine
#### Eemaldada keelendid, mis ei ole ÜSis

Kuna lõpuks on vaja potentsiaalsed kandidaadid siduda ÜSi tähendustega, on mõtet tegelda vaid nende keelenditega, mis ÜSis olemas on. Samas tähendab see paraku, et kaduma läheb hulk keelendeid, mis on saadud korpustest.

In [436]:
def load_ys_file_to_set(filename):
    ys_set = set()
    with open(filename, 'r', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            ys_set.add(row[0])
    return ys_set

def is_word_in_ys_file(word, ys_set):
    if word in ys_set:
        return True
    else:
        return False

ys_set = load_ys_file_to_set(ys)

with open(base_data, 'r', encoding='utf-8') as f:
    data = json.load(f)

removed_items = []
filtered_data = []

for d in data:
    headword_value = d["headwordValue"]

    if is_word_in_ys_file(headword_value, ys_set):
        filtered_data.append(d)
    else:
        removed_items.append(d)

#### Kokkuvõte

In [437]:
columns = ['Tingimus', 'Keelendeid ÜSis', 'Näited']
random_ys_values = random.sample(filtered_data, 10)
headword_ys_values = [item["headwordValue"] for item in random_ys_values]
ys_words = ', '.join(headword_ys_values)

random_not_ys_values = random.sample(removed_items, 10)
headword_not_ys_values = [item["headwordValue"] for item in random_not_ys_values]
not_ys_words = ', '.join(headword_not_ys_values)
    
ys_data = [
    ["Keelend ÜSis olemas", len(filtered_data), ys_words],
    ["Keelend puudub ÜSis", len(removed_items), not_ys_words]
]

pd.set_option('display.max_colwidth', None)

df_ys = pd.DataFrame(ys_data, columns=columns)

styled_df_ys = df_ys.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}]).hide()

styled_df_ys

Tingimus,Keelendeid ÜSis,Näited
Keelend ÜSis olemas,109464,"värviergas, väsimustugevus, õhusoojus, vankumatus, kirikuvõim, karbonaad, pinnapealsus, lipiid, valitsusside, düsenteeriaepideemia"
Keelend puudub ÜSis,1811922,"zygoptera, acocleani, defferre, tindero, kübaratäis, ricks, jäikusväärtus, asendaminesa, noraku, lisapunker"


Alles jäänud keelendid salvestatakse ajutisse JSON faili `temp_filtered_data.json`.

In [438]:
with open(f'output/temp_filtered_data.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False)

Eemaldatud keelendid salvestatakse JSON faili `words_not_in_ys.json`.

In [439]:
with open(f'output/words_not_in_ys.json', 'w', encoding='utf-8') as f:
    json.dump(removed_items, f, ensure_ascii=False, indent=2)

### Andmete saniteerimine
Jätkatakse faili `temp_filtered_data.json` andmete saniteerimisega.

In [440]:
with open(f'output/temp_filtered_data.json', 'r', encoding='utf-8') as f:
    filtered_data = json.load(f)

#### Sagedusloendid

Laaditakse sagedusloendid. Fail koosneb kolmest loetelust:

- [Ngramid Peter Norvigi lehelt](https://norvig.com/ngrams/count_1w.txt)
- [Sagedusloend Peter Norvigi lehelt](<https://norvig.com/ngrams/word.list>)
- [Collins Scrabble Words (2019)](<https://drive.google.com/file/d/1oGDf1wjWp5RF_X9C7HoedhIWMh5uJs8s/view>)

In [441]:
with open('input/sagedusloendid.txt', 'r', encoding='utf-8') as f:
    frequency_words = set(line.strip() for line in f.readlines())

#### Must nimekiri

Korpuste parsimisel sattus kandidaatideks eessõnu, artikleid jmt, mis polnud siinkohal olulised (nt *to*, *in*, *the*, *at*, *are*). Koos Kristina Koppeliga sai neist koostatud must nimekiri, mis nüüd laaditakse.

In [442]:
with open('input/blacklist.txt', 'r', encoding='utf-8') as f:
    words_to_be_ignored = set(line.strip() for line in f.readlines())

#### Üleliigsete inglise tõlkevastete eemaldamine ja vajadusel kohendamine

Seejärel käiakse läbi massiivis olevad sõnastikud ja kontrollitakse tõlkevastete kandidaatide vastavust mitmesugustele reeglitele. 

Reeglitele vastavust kontrollitakse järjestikku nii, et kui keelend mingit reeglit rikub, katkestatakse tsükkel ja asutakse järgmist keelendit kontrollima. Kui keelend vastab reeglile, siis kontrollitakse vastavust järgmisele reeglile, kuni tsükkel läbi saab.

Kui tõlkevaste kandidaat ei vasta reeglitele, siis lisatakse see massiivi `words_to_remove`. Kui ühe sõnastiku kõik tõlkevastete kandidaadid on läbi käidud, eemaldatakse massiivist synCandidateWords need keelendid, mis on massiivis `words_to_remove`. Nii on tsükli lõpuks massiivis `sanitized_data` sõnastikud, mis läbisid reeglite kontrolli. Paralleelselt koostatakse sõnastikud eestikeelsest keelendist ja inglise vastete kandidaatidest, mis reeglitele ei vastanud. Need sõnastikud salvestatakse massiivina `removed_items`. 

##### Reeglid

1. Kui tõlkevaste kandidaat on ühesõnaline (ei sisalda tühikut ega sidekriipsu) ja selle kaal on suurem kui 0,8, siis jäetakse see alles.
2. Kui tõlkevaste kandidaat on ühesõnaline (ei sisalda tühikut ega sidekriipsu) ja kaal ei ole suurem kui 0,8, siis jäetakse see alles, kui sõna on sagedusloendites. Kui ei ole, eemaldatakse.
3. Kui tõlkevaste kandidaat on mustas nimekirjas, aga selle definitsioon on pärit KDictionaryst, siis see keelend jääb alles.
4. Kui tõlkevaste kandidaat on mustas nimekirjas, aga sel pole definitsiooni või definitsioon pole pärit KDictionaryst, siis see keelend eemaldatakse.
5. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artikliga 'the ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on  samasugune tõlkevaste kandidaat ilma artiklita 'the ', siis eemaldatakse artikliga variant ja jäetakse alles artiklita keelend.
6. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artilkiga 'the ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on samasugune tõlkevaste kandidaat ilma artiklita 'the ', siis eemaldatakse sõna algusest 'the '.
7. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artikliga 'a ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on  samasugune tõlkevaste kandidaat ilma artiklita 'a ', siis eemaldatakse artikliga variant ja jäetakse alles artiklita keelend.
8. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artilkiga 'a ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on samasugune tõlkevaste kandidaat ilma artiklita 'a ', siis eemaldatakse sõna algusest 'a '.
9. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artikliga 'an ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on  samasugune tõlkevaste kandidaat ilma artiklita 'an ', siis eemaldatakse artikliga variant ja jäetakse alles artiklita keelend.
10. Kui väiketähtede kujule viidud tõlkevaste kandidaat algab artilkiga 'an ' ja samas massiivis pole sõnastikku, mille võtme 'value' väärtus on samasugune tõlkevaste kandidaat ilma artiklita 'an ', siis eemaldatakse sõna algusest 'an '.
11. Kui tõlkevaste kandidaat on sõne, mis koosneb tühikuga eraldatud samasugustest sõnedest, siis jäetakse neist sõnedest alles vaid üks (nt 'hello hello' asemele jääb 'hello').
12. Kui tõlkevaste kandidaat algab kirjavahemärgi (!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~) ja tühikuga, siis eemaldatakse see keelend.
13. Kui tõlkevaste kandidaat sisaldab numbrit, siis see keelend eemaldatakse.
14. Kui tõlkevaste kandidaat sisaldab "'s" või " 's", siis see keelend eemaldatakse.

In [443]:
sanitized_data = filtered_data.copy()
removed_items = []

for d in sanitized_data:
    headword_value = d["headwordValue"]
    words_to_remove = []
    removed_items_dict = {"headwordValue": headword_value, "synCandidateWords": []}

    for item in d["synCandidateWords"]:
        if ' ' not in item['value'] and '-' not in item['value']:
            # 1. reegel:
            if item['weight'] > 0.8:
                continue
            # 2. reegel:
            if item["value"] not in frequency_words:
                words_to_remove.append(item)
                removed_items_dict["synCandidateWords"].append(item)

        if item['value'] in words_to_be_ignored:
            has_kdictionary = False
            for definition in item['definitions']:
                # 3. reegel:
                for sourceLink in definition['sourceLinks']:
                    if sourceLink['value'] == 'KDictionary':
                        has_kdictionary = True
                        break
                if has_kdictionary:
                    break
            # 4. reegel:
            if not has_kdictionary:
                words_to_remove.append(item)
                removed_items_dict["synCandidateWords"].append(item)

        if item['value'].lower().startswith('the '):
            word_without_article = item['value'][4:]
            # 5. reegel:
            if any(word_without_article == word['value'] for word in d["synCandidateWords"]):
                words_to_remove.append(item)
                removed_items_dict["synCandidateWords"].append(item)
            else:
                # 6. reegel:
                item['value'] = word_without_article

        if item['value'].lower().startswith('a '):
            word_without_article = item['value'][2:]
            # 7. reegel:
            if any(word_without_article == word['value'] for word in d["synCandidateWords"]):
                words_to_remove.append(item)
                removed_items_dict["synCandidateWords"].append(item)
            else:
                # 8. reegel:
                item['value'] = word_without_article

        if item['value'].lower().startswith('an '):
            word_without_article = item['value'][3:]
            # 9. reegel:
            if any(word_without_article == word['value'] for word in d["synCandidateWords"]):
                words_to_remove.append(item)
                removed_items_dict["synCandidateWords"].append(item)
            else:
                # 10. reegel:
                item['value'] = word_without_article

        # 11. reegel
        words = item['value'].split()
        if len(words) > 1 and all(word == words[0] for word in words):
            words_to_remove.append(item)
            removed_items_dict["synCandidateWords"].append(item)

        # 12. reegel:
        if any(item['value'].startswith(punct + " ") for punct in string.punctuation):
            words_to_remove.append(item)
            removed_items_dict["synCandidateWords"].append(item)

        # 13. reegel:
        if any(char.isdigit() for char in item['value']):
            words_to_remove.append(item)
            removed_items_dict["synCandidateWords"].append(item)

        # 14. reegel:
        if " 's" in item['value']:
            words_to_remove.append(item)
            removed_items_dict["synCandidateWords"].append(item)

    for item in words_to_remove:
        if item in d["synCandidateWords"]:
            d["synCandidateWords"].remove(item)

    if removed_items_dict["synCandidateWords"]:
        removed_items.append(removed_items_dict)

Eestikeelsed keelendid koos võimalike ingliskeelsete tõlkevastete kandidaatidega, mis alles jäid, salvestatakse faili.

In [444]:
with open('output/final_sanitized_data.json', 'w', encoding='utf-8') as f:
    json.dump(sanitized_data, f, ensure_ascii=False, indent=2)

Sõnastikud sobimatute ingliskeelsete vastetega salvestatakse faili.

In [445]:
with open('output/final_removed_words.json', 'w', encoding='utf-8') as f:
    json.dump(removed_items, f, ensure_ascii=False, indent=2)

##### Kokkuvõte

In [446]:
columns = ['Keelendi tüüp', 'Algandmestik', 'Ainult ÜSi keelenditega', 'Lõpptulemus', 'Vahe']

count_data = 0
for dict_item in data:
    count_data += len(dict_item['synCandidateWords'])

count_filtered_data = 0
for i in filtered_data:
    count_filtered_data += len(i['synCandidateWords'])

count_sanitized_data = 0
for i in sanitized_data:
    count_sanitized_data += len(i['synCandidateWords'])
    
data = [
    ["Eestikeelne keelend ('headwordValue')", len(data), len(filtered_data), len(sanitized_data), len(sanitized_data) - len(data)],
    ["Ingliskeelne tõlkevaste kandidaat ('value')", count_data, count_filtered_data, count_sanitized_data, count_sanitized_data - count_data]
]

df = pd.DataFrame(data, columns=columns)

styled_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}]).hide()

styled_df

Keelendi tüüp,Algandmestik,Ainult ÜSi keelenditega,Lõpptulemus,Vahe
Eestikeelne keelend ('headwordValue'),1921386,109464,109464,-1811922
Ingliskeelne tõlkevaste kandidaat ('value'),4199247,994220,994220,-3205027


## Tähenduste lisamine WordNetist

Tähenduse lisamiseks inglise vastete kandidaatidele sai proovitud NLTK teegi kaudu saadavalolevat WordNeti.

Laaditakse vajalikud teegid.

In [447]:
import nltk
from nltk.corpus import wordnet as wn
from pprint import pprint

Kasutatakse saniteeritud andmestikku.

In [448]:
with open('output/final_sanitized_data.json', 'r', encoding='utf-8') as file:
    final_sanitized_data = json.load(file)

Andmestiku sõnaliikide lühendite vastavusse viimine WordNeti omadega. WordNeti klassifikatsioonis märgib _a_ 'adjectives' ja _s_ 'satellite adjectives'.

In [449]:
pos_mapping = {
    'n': 's',
    'v': 'v',
    'a': 'adj',
    's': 'adj',
    'r': 'adv'
}

Funktsioon keelendi definitsiooni saamiseks WordNetist.

In [450]:
def get_wordnet_definitions(word):
    synsets = wn.synsets(word)
    definitions_by_pos = {}
    for synset in synsets:
        pos = synset.pos()
        if pos not in definitions_by_pos:
            definitions_by_pos[pos] = []
        definitions_by_pos[pos].append(synset.definition())
    return definitions_by_pos

Funktsioon `get_wordnet_definitions` tagastab sõnastiku, milles võtmeteks on sõnaliikide lühendid ja väärtusteks definitsioonid. Näide:

In [451]:
pprint(get_wordnet_definitions('social'))

{'a': ['relating to human society and its members',
       'living together or enjoying life in communities or organized groups',
       'relating to or belonging to or characteristic of high society'],
 'n': ['a party of people assembled to promote sociability and communal '
       'activity'],
 's': ['composed of sociable people or formed for the purpose of sociability',
       'tending to move or live together in groups or colonies of the same '
       'kind',
       'marked by friendly companionship with others']}


Käiakse läbi saniteeritud andmestiku inglise tõlkevastete kandidaadid ja lisatakse definitsioone, kui neid on võimalik WordNetist saada. 
Definitsiooni allikaks määratakse NLTK WordNet ja allika ID-ks kohatäiteks valitud ID. Kui tõlkevaste kandidaadil on kaal olemas, kasutatakse seda, kui ei, on vaikimisi kaalu väärtuseks 1.0. Lõpptulemus on massiiv sõnastikest, kus iga sõnastik koosneb eesti keelendist ja inglise keele tõlkevastete kandidaatidest koos definitsioonidega. Kui kandidaadid on ühest sõnaliigist, on need ühes sõnastikus.

In [452]:
def process_data(data):
    for entry in data:
        syn_candidate_words = entry.get('synCandidateWords', [])
        new_syn_candidate_words = []

        for syn_word in syn_candidate_words:
            word_value = syn_word.get('value')
            wordnet_definitions_by_pos = get_wordnet_definitions(word_value)  # Assuming this function is defined elsewhere

            for pos, definitions in wordnet_definitions_by_pos.items():
                mapped_pos = pos_mapping.get(pos, pos)  # Assuming pos_mapping is defined elsewhere
                for defn in definitions:
                    new_word_entry = {
                        "value": word_value,
                        "lang": "eng",
                        "weight": syn_word.get('weight', 1.0),
                        "posCodes": [mapped_pos],
                        "definitions": [{"value": defn, "sourceLinks": [{"sourceId": 84967, "value": "NLTK WordNet"}]}],
                        "usages": syn_word.get('usages', [])
                    }
                    new_syn_candidate_words.append(new_word_entry)

            for existing_defn in syn_word.get('definitions', []):
                new_word_entry = {
                    "value": word_value,
                    "lang": "eng",
                    "weight": syn_word.get('weight', 1.0),
                    "posCodes": syn_word.get('posCodes', []),
                    "definitions": [existing_defn],
                    "usages": syn_word.get('usages', [])
                }
                new_syn_candidate_words.append(new_word_entry)

        merged_by_posCodes = {}

        for d in new_syn_candidate_words:
            posCodes_key = tuple(d['posCodes'])
            if posCodes_key in merged_by_posCodes:
                merged_by_posCodes[posCodes_key]['definitions'] += d['definitions']
            else:
                merged_by_posCodes[posCodes_key] = d

        merged_dicts = list(merged_by_posCodes.values())

        entry['synCandidateWords'] = merged_dicts

    return data

processed_data = process_data(final_sanitized_data)

Näide esimesest eesti keelendist, mille inglise keele tõlkevaste kandidaadile on lisatud WordNetist definitsioon.

In [453]:
for item in processed_data:
    for syn_word in item['synCandidateWords']:
        found = False
        for definition in syn_word['definitions']:
            for source_link in definition['sourceLinks']:
                if source_link['value'] == 'NLTK WordNet':
                    print(json.dumps(item, indent=2, ensure_ascii=False))
                    found = True
                    break
            if found:
                break
        if found:
            break
    if found:
        break

{
  "headwordValue": "sotsiaalhoolekanne",
  "headwordLang": "est",
  "synCandidateDatasetCode": "ingitermax",
  "synCandidateWords": [
    {
      "value": "social work",
      "lang": "eng",
      "weight": 0.9,
      "posCodes": [
        "s"
      ],
      "definitions": [
        {
          "value": "work which deals with the care of people in a community, especially of the poor, under-privileged etc.",
          "sourceLinks": [
            {
              "sourceId": 20647,
              "value": "KDictionary"
            }
          ]
        },
        {
          "value": "a party of people assembled to promote sociability and communal activity",
          "sourceLinks": [
            {
              "sourceId": 84967,
              "value": "NLTK WordNet"
            }
          ]
        },
        {
          "value": "governmental provision of economic assistance to persons in need",
          "sourceLinks": [
            {
              "sourceId": 84967,
              

Tulemus salvestatakse faili.

In [ ]:
with open('output/data_with_wordnet_definitions.json', 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, indent=4, ensure_ascii=False)